This model generates a projected value for a given statistic by using a weighted mean of the previous three seasons for each player, with weights optimized using SciPy to minimize error. It also distinguishes between cumulative and rate statistics, calculating cumulative stats per plate appearance for improved accuracy. No projections are made for first-year players.

# Import Packages and Data

In [22]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [23]:
# Load the data from a CSV file into a DataFrame
df = pd.read_csv('../Resources/properly_formatted_data.csv')

In [24]:
# Seperate stats by cumulative and rate
pa_list = ['H', '1B', '2B', '3B', 'HR', 'R', 'RBI', 'BB', 'SH', 'SB', 'L-WAR', 'wRC', 'WAR']
non_pa_list = ['G', 'AB', 'PA', 'AVG', 'BB%', 'OBP', 'SLG', 'OPS', 'ISO', 'wOBA', 'wRAA', 'wRC+', 'BB%+']

# Automate

In [25]:
# Calculate a weighted mean projection based on three years of data.
# Normalize the weights to ensure they sum to 1 before returning the result.
def projection(weights, year1, year2, year3):
    weighted_mean = weights[0] * year1 + weights[1] * year2 + weights[2] * year3
    weighted_mean /= np.sum(weights)  # Normalize the weights to sum to 1
    return weighted_mean

In [26]:
def objective(params, df):
    weights = params[:3]  # First 3 params are the weights for the years
    
    total_rmse = 0  # Sum of RMSE for all players
    
    # Loop through each player and compute the error
    for _, row in df.iterrows():
        # Access columns by their positional index
        year1_stat, year2_stat, year3_stat = row.iloc[1], row.iloc[2], row.iloc[3]

        observed_stat = row.iloc[0]
        
        # Compute projected stats
        projected_stat = projection(weights, year1_stat, year2_stat, year3_stat)
        
        
        # Compute RMSE for stat
        rmse_stat = np.sqrt(np.mean((projected_stat - observed_stat) ** 2))
        
        
        total_rmse += rmse_stat
    
    return total_rmse  # Total RMSE for the entire dataset

In [27]:
def calculate_projected_stat(row):
    # Get the previous 3 years' stat values
    previous_stat_values = row.iloc[1:4].values
    
    # Filter out NaN values (which represent missing data for second and third year players) and their corresponding weights
    valid_values = [val for val, weight in zip(previous_stat_values, optimized_weights) if not np.isnan(val)]
    valid_weights = [weight for val, weight in zip(previous_stat_values, optimized_weights) if not np.isnan(val)]
    
    # Check if no valid data exists
    if len(valid_values) == 0:
        return np.nan  # Return NaN if no previous value is available (rookie season)
    
    # Calculate the weighted average of the valid previous stat values
    weighted_sum = sum(val * weight for val, weight in zip(valid_values, valid_weights))
    weighted_avg = weighted_sum / sum(valid_weights)
    
    return weighted_avg

In [28]:
# Iterate through each statistic in the 'non_pa_list' for projection.
for stat in non_pa_list:
    # For each stat, create a subset of the DataFrame containing the current season's stat
    # and the stats from the previous three seasons (e.g., '1Prev_stat', '2Prev_stat', '3Prev_stat').
    training_df = df[[stat, f'1Prev_{stat}', f'2Prev_{stat}', f'3Prev_{stat}']]

    # Copy the data for future use and drop any rows with missing (NaN) values.
    training_df_original = training_df.copy()
    training_df = training_df.dropna()

    # Use an initial guess of equal weights (0.33) for the three seasons to start the optimization.
    initial_guess = [0.33, 0.33, 0.33]

    # Apply the Nelder-Mead optimization method to minimize the objective function and find the best weights.
    result = minimize(objective, initial_guess, args=(training_df,), method='Nelder-Mead')

    # Extract the optimized weights for the three seasons and print them for verification.
    optimized_weights = result.x[:3]
    print("Optimized Weights:", optimized_weights)

    # Apply the 'calculate_projected_stat' function to the cleaned data to generate the projected stat
    # and add it as a new column in the DataFrame
    df[f'Projected_{stat}'] = training_df_original.apply(calculate_projected_stat, axis=1)

Optimized Weights: [0.55706882 0.14237771 0.0636827 ]
Optimized Weights: [0.60607341 0.12210693 0.06075946]
Optimized Weights: [0.65158376 0.12981028 0.0617733 ]
Optimized Weights: [0.42961582 0.31509783 0.20345475]
Optimized Weights: [0.43566675 0.30478457 0.1969243 ]
Optimized Weights: [0.430343   0.30751144 0.19723082]
Optimized Weights: [0.47859045 0.30625046 0.20317302]
Optimized Weights: [0.45856742 0.31210143 0.20945251]
Optimized Weights: [0.5024387  0.2972551  0.19577622]
Optimized Weights: [0.43250237 0.3035576  0.19775468]
Optimized Weights: [0.47453558 0.27859936 0.22395609]
Optimized Weights: [0.43523122 0.30351999 0.20522359]
Optimized Weights: [0.44503495 0.30780076 0.19854949]


# Adding per PA

In [31]:
def calculate_projected_stat_per_pa(row):
    # Get the previous 3 years' stat values
    previous_stat_values = row.iloc[1:4].values
    
    # Filter out NaN values (which represent missing data for second and third year players) and their corresponding weights
    valid_values = [val for val, weight in zip(previous_stat_values, optimized_weights) if not np.isnan(val)]
    valid_weights = [weight for val, weight in zip(previous_stat_values, optimized_weights) if not np.isnan(val)]
    
    # Check if no valid data exists
    if len(valid_values) == 0:
        return np.nan  # Return NaN if no previous value is available (rookie season)
    
    # Calculate the weighted average of the valid previous stat values
    weighted_sum = sum(val * weight for val, weight in zip(valid_values, valid_weights))
    weighted_avg = weighted_sum / sum(valid_weights)

    # Multiply the expecteted per PA stat by projected PA to get an estimate for projected cumlative stat
    regressed_avg = weighted_avg * row.iloc[4]
    
    return regressed_avg

In [32]:
# Iterate through each statistic in the 'non_pa_list' for projection.
for stat in pa_list:
    # Calculate per plate appearance (PA) values for the current and previous three seasons
    # by dividing the stat for each season by the corresponding projected or actual PA.
    df[f'{stat}/PA'] = df[stat] / df['Projected_PA']
    df[f'1Prev_{stat}/PA'] = df[f'1Prev_{stat}'] / df['1Prev_PA']
    df[f'2Prev_{stat}/PA'] = df[f'2Prev_{stat}'] / df['2Prev_PA']
    df[f'3Prev_{stat}/PA'] = df[f'3Prev_{stat}'] / df['3Prev_PA']

    # Create a 'training_df' with the calculated per-PA values and the projected PA 
    # to be used in the optimization or further analysis.
    training_df = df[[f'{stat}/PA', f'1Prev_{stat}/PA', f'2Prev_{stat}/PA', f'3Prev_{stat}/PA', 'Projected_PA']]

    # Copy the data for future use and drop any rows with missing (NaN) values.
    training_df_original = training_df.copy()
    training_df = training_df.dropna()

    # Use an initial guess of equal weights (0.33) for the three seasons to start the optimization.
    initial_guess = [0.33, 0.33, 0.33]

    # Apply the Nelder-Mead optimization method to minimize the objective function and find the best weights.
    result = minimize(objective, initial_guess, args=(training_df,), method='Nelder-Mead')

    # Extract the optimized weights for the three seasons and print them for verification.
    optimized_weights = result.x[:3]
    print("Optimized Weights:", optimized_weights)

    # Apply the 'calculate_projected_stat_per_pa' function to the cleaned data to generate the projected stat
    # and add it as a new column in the DataFrame
    df[f'Projected_{stat}'] = training_df_original.apply(calculate_projected_stat_per_pa, axis=1)

Optimized Weights: [0.51078732 0.25702032 0.17601161]
Optimized Weights: [0.49111432 0.30726938 0.20644621]
Optimized Weights: [0.49227301 0.26364831 0.17673368]
Optimized Weights: [0.55075862 0.29879894 0.13028598]
Optimized Weights: [0.51864579 0.24960802 0.14697175]
Optimized Weights: [0.5726461  0.28837273 0.11705846]
Optimized Weights: [0.50856386 0.28501604 0.16570355]
Optimized Weights: [0.49846368 0.31202633 0.18759177]
Optimized Weights: [0.42385782 0.30626452 0.21442836]
Optimized Weights: [0.53230826 0.23240628 0.12287041]
Optimized Weights: [0.43577103 0.31527484 0.18602095]
Optimized Weights: [0.49173994 0.30143547 0.20096576]
Optimized Weights: [0.43624589 0.31621958 0.18578682]


# Organize and Export

In [ ]:
columns_to_keep = ["IDfg", "Name", "Season", "Team", "Age"]

In [ ]:
df = df.drop(columns=df.filter(like='Prev').columns)

In [ ]:
df = df.drop(columns=df.filter(like='/PA').columns)

In [ ]:
# Create a list of stats (e.g., 'H', 'HR', etc.)
stats = pa_list + non_pa_list

# Create a new column order
ordered_columns = []
for stat in stats:
    ordered_columns.append(stat)  # Add actual stat
    projected_col = f"Projected_{stat}"
    if projected_col in df.columns:
        ordered_columns.append(projected_col)  # Add projected stat if it exists

In [ ]:
columns_to_keep = columns_to_keep + ordered_columns
df = df.loc[:, columns_to_keep]

In [ ]:
df.head()

In [ ]:
# Export to a specific folder
df.to_csv('../Projection_Results/model_three.csv', index=False)